#### 部署已训练好的多个img2vec模型

In [1]:
import tarfile
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('img2vec1.pth')
    f.add('img2vec2.pth')

In [142]:
!tar -tf ./model.tar.gz

sh: 0: getcwd() failed: No such file or directory
img2vec1.pth
img2vec2.pth


In [3]:
import boto3
import sagemaker
from time import gmtime, strftime
from sagemaker.pytorch.model import PyTorchModel

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
boto3.Session().resource('s3').Bucket(bucket).Object('lb-mme/model.tar.gz').upload_file('model.tar.gz')

In [353]:
sess

In [143]:
from sagemaker.serializers import IdentitySerializer, JSONSerializer

_time_tag = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_s3_path = 's3://{}/lb-mme/model.tar.gz'.format(bucket)

pytorch_serving_model = PyTorchModel(model_data=model_s3_path,
                                     framework_version='1.6',
                                     role=role,
                                     entry_point='inference.py',
                                     sagemaker_session=sess,
                                     py_version='py3')

_endpoint_name = 'lb-mme-' + _time_tag

predictor = pytorch_serving_model.deploy(endpoint_name = _endpoint_name, instance_type='ml.g4dn.xlarge', initial_instance_count=1, serializer=JSONSerializer())

-------!

In [148]:
import json
import io
import base64

"""import io

with io.BytesIO() as output:
    img_test.save(output, format="png")
    contents = output.getvalue()"""

with open('dog.png', 'rb') as f:
    img_bytes = f.read()
    
img_base = base64.encodebytes(img_bytes).decode()
# stream = io.BytesIO(base64.b64decode(img_base.encode()))
# img = Image.open(stream).convert('RGB')
img_dic = {'target_model': 2, 'img_base': img_base}
# img_json = json.dumps({'target_model': 1, 'img_base': img_base})
# json.loads(img_json)['img_base'].encode()

In [99]:
# stream = io.BytesIO(img_bytes)

In [102]:
# img = Image.open(stream).convert('RGB')

In [103]:
# t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

In [104]:
# t_img

tensor([[[[ 0.9132,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 0.9132,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 0.9132,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [ 0.2796,  1.5639,  1.4098,  ...,  2.1975,  2.2147,  2.2489],
          [ 0.2796,  1.5468,  1.5810,  ...,  2.1975,  2.2318,  2.2489],
          [ 0.2624,  1.3927,  1.5125,  ...,  2.1975,  2.2489,  2.2489]],

         [[ 1.0805,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 1.0805,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 1.0805,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          ...,
          [ 0.0476,  1.1681,  0.9930,  ...,  2.3235,  2.4286,  2.4286],
          [ 0.0476,  1.1331,  1.1506,  ...,  2.3585,  2.4286,  2.4286],
          [ 0.0126,  0.9930,  1.0980,  ...,  2.3761,  2.4286,  2.4286]],

         [[ 1.3154,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
          [ 1.3154,  2.6400,  

In [153]:
res = predictor.predict(img_dic)

In [155]:
import numpy as np

v = res.tolist()['result'][0]

In [156]:
len(v)

1000

In [ ]:
v